# Using Wallaroo Shadow Deployment in Computer Vision

In this tutorial we will explore using Wallaroo's Shadow Deployment Technology to safely improve your Object Detectors in a production environments.

In step 3 we learned how to deploy a object detector into a Wallaroo pipeline, run inference, and draw the detected objects, the bounding boxes, the classifications, and the confidence of the classifications on the sample image.

Now we will expand on that lesson by using 2 object detectors.  The mobilnet object detector is the control and the faster-rcnn object detector is the challenger.  Lets compare the inferencing results when we feed our shadow deployment pipeline an image.

In [ ]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import pandas as pd
from CVDemoUtils import CVDemo


In [ ]:
wl = wallaroo.Client()

In [ ]:
control =  wl.upload_model('mobilenet', 'models/mobilenet.pt.onnx')

In [ ]:
challenger = wl.upload_model('resnet50', 'models/frcnn-resnet.pt.onnx')

In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("8Gi").build()

In [ ]:
pipeline = wl.build_pipeline("shadow-pp")
pipeline.add_shadow_deploy(control, [challenger])
pipeline.deploy(deployment_config = deployment_config)

In [ ]:
time.sleep(5) # needed to allow the pipeline to settle in.

## Test the shadow deployment by running inference on a sample image

### Prepare input image

Next we will load a sample image and resize it to the width and height required for the object detector.

We will convert the image to a numpy ndim array and add it do a dictionary

In [ ]:

#imagePath = 'images/input/example/example_01.jpg'
imagePath = 'data/images/input/example/store-front.png'

# The image width and height needs to be set to what the model was trained for.  In this case 640x480.
cvDemo = CVDemo()

# The size the image will be resized to meet the input requirements of the object detector
width = 640
height = 480
tensor, controlImage = cvDemo.loadImageAndResize(imagePath, width, height)
challengerImage = controlImage.copy()

# get npArray from the tensorFloat
npArray = tensor.cpu().numpy()

#creates a dictionary with the wallaroo "tensor" key and the numpy ndim array representing image as the value.
dictData = {"tensor": npArray.tolist()}


# Run inference using the Shadow Deployment using the SDK 

Now lets have the model detect the objects on the image by running inference and extracting the results 

In [ ]:

startTime = time.time()
infResults = pipeline.infer(dictData)
endTime = time.time()

results = infResults[0].raw
results['original_data'] = None  # We are removing the input image json.  Not needed
results

## Extracting the Control inference information and putting it in a table

We will create a dataframe with columns representing the classification, confidence, and bounding boxes of the objects identified.

Lets handle extracting the bounding boxes from the inference results for the objeects detected.  Once extracted from the results we will want to reshape the flattened array into an array with 4 elements (x,y,width,height)

In [ ]:
df = pd.DataFrame(columns=['classification','confidence','x','y','width','height'])
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:.2%}'.format

# Points to where all the inference results are
outputs = results['outputs']
shadow_data = results['shadow_data']

controlBoxes = outputs[0]

# reshape this to an array of bounding box coordinates converted to ints
boxList = controlBoxes['Float']['data']
boxA = np.array(boxList)
controlBoxes = boxA.reshape(-1, 4)
controlBoxes = controlBoxes.astype(int)

df[['x', 'y','width','height']] = pd.DataFrame(controlBoxes)

controlClasses = outputs[1]['Int64']['data']
controlConfidences = outputs[2]['Float']['data']

idx = 0 
cocoClasses = cvDemo.getCocoClasses()
for idx in range(0,len(controlClasses)):
    df['classification'][idx] = cocoClasses[controlClasses[idx]] # Classes contains the 80 different COCO classificaitons
    df['confidence'][idx] = controlConfidences[idx]
df



### Display the Control Results


Here we will use the Wallaroo CVDemo helper class to draw the control model results on the image

In [ ]:

results = {
    'model_name' : control.name(),
    'pipeline_name' : pipeline.name(),
    'width': width,
    'height': height,
    'image' : controlImage,
    'boxes' : controlBoxes,
    'classes' : controlClasses,
    'confidences' : controlConfidences,
    'confidence-target' : 0.9,
    'color':CVDemo.RED, # color to draw bounding boxes and the text in the statistics
    'inference-time': (endTime-startTime),
    'onnx-time' : 0,                
}
cvDemo.drawAndDisplayDetectedObjectsWithClassification(results)



## Extracting the Challenger inference information and putting it in a table

Lets handle extracting the bounding boxes from the inference results for the objeects detected.  Once extracted from the results we will want to reshape the flattened array into an array with 4 elements (x,y,width,height)

In [ ]:
challengerBoxes = shadow_data['resnet50'][0]
# reshape this to an array of bounding box coordinates converted to ints
boxList = challengerBoxes['Float']['data']
boxA = np.array(boxList)
challengerBoxes = boxA.reshape(-1, 4)
challengerBoxes = challengerBoxes.astype(int)

challengerDf = pd.DataFrame(columns=['classification','confidence','x','y','width','height'])
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:.2%}'.format

challengerDf[['x', 'y','width','height']] = pd.DataFrame(challengerBoxes)
#pd.options.display.float_format = '{:.2%}'.format
challengerClasses = shadow_data['resnet50'][1]['Int64']['data']
challengerConfidences = shadow_data['resnet50'][2]['Float']['data']

idx = 0 
for idx in range(0,len(challengerClasses)):
    challengerDf['classification'][idx] = cvDemo.CLASSES[challengerClasses[idx]] # Classes contains the 80 different COCO classificaitons
    challengerDf['confidence'][idx] = challengerConfidences[idx]
challengerDf

### Display the Challenger Results


Here we will use the Wallaroo CVDemo helper class to draw the challenger model results on the input image

In [ ]:

blue = (255, 0, 0)
results = {
    'model_name' : challenger.name(),
    'pipeline_name' : pipeline.name(),
    'width': width,
    'height': height,
    'image' : challengerImage,
    'boxes' : challengerBoxes,
    'classes' : challengerClasses,
    'confidences' : challengerConfidences,
    'confidence-target' : 0.90,
    'color':CVDemo.BLUE, # color to draw bounding boxes and the text in the statistics
    'inference-time': (endTime-startTime),
    'onnx-time' : 0,                
}
cvDemo.drawAndDisplayDetectedObjectsWithClassification(results)

In [17]:
pipeline.undeploy()
#for d in wl.list_deployments():
#    d.undeploy()
    

.................. ok


name,shadow-pp
created,2022-10-29 15:29:46.904026+00:00
last_updated,2022-12-14 13:00:36.357872+00:00
deployed,False
tags,
steps,mobilenet


### Conclusion

Notice the difference in the control confidence and the challenger confidence.  <b>Clearly we can see in this example the challenger resnet50 model is performing better than the control mobilenet model</b>.  This is likely due to the fact that frcnn resnet50 model is a 2 stage object detector vs the frcnn mobilenet is a single stage detector.
